In [13]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [14]:
data_directory =  '/opt/ml/input/data'

TRAIN_CSV = f"{data_directory}/train.csv"
TRAIN_PATH = f"{data_directory}/train/images"
TEST_PATH = f"{data_directory}/eval/images"

In [15]:
import cv2

import torch
import torchvision
from torchvision.transforms import transforms
from torchvision.transforms import ToPILImage
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data_directory, mode, transforms = None, resize = False):
        super().__init__()
        # self.data_directory = data_directory
        self.TRAIN_CSV = f"{data_directory}/train.csv"
        self.TRAIN_PATH = f"{data_directory}/train/images"
        self.TEST_PATH = f"{data_directory}/eval/images"
        self.transforms = transforms
        self.mode = mode
        # self.train_info = train_info

        # get the path of the image_path
        data_path = []
        labels = []
        if mode == 'train':
            people = sorted([who for who in os.listdir(TRAIN_PATH) if who[0] != '.'])
            for who in people:
                for mask_type in sorted([mask_type for mask_type in os.listdir(f"{TRAIN_PATH}/{who}") if mask_type[0] != '.']):
                    data_path.append( os.path.join(f"{TRAIN_PATH}/{who}/{mask_type}") )

                    _, gender, race, age = who.split('_')
                    mask, _ = mask_type.split('.')

                    score = 0
                    # gender
                    if gender == 'female': 
                        score += 3

                    # ages
                    age = int(age)
                    if age >= 30 and age < 60 :
                        score += 1
                    elif age >= 60:
                        score += 2

                    # mask type
                    if mask == "incorrect_mask" :
                        score += 6
                    elif mask == 'normal':
                        score += 12

                    labels.append(score)

        elif mode == 'eval':
            people = sorted([who for who in os.listdir(TEST_PATH) if who[0] != '.'])
            for who in people:
                # for mask_type in sorted([mask_type for mask_type in os.listdir(f"{TEST_PATH}/{who}") if mask_type[0] != '.']):
                    data_path.append( os.path.join(f"{TEST_PATH}/{who}") )
                
        self.image_path = data_path
        self.labels = labels
                        
    def __getitem__(self, idx):
        img_path = self.image_path[idx]
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)

        if self.transforms is not None :
            img = self.transforms(img)

        if self.mode == 'test':
            return img
        
        label = self.labels[idx]
        return img, label

    
    def __len__(self):
        return len(self.img_path)